In [11]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from dotenv import load_dotenv
import os
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
load_dotenv()

# Initialize FastAPI app
app = FastAPI()

# Initialize components globally (loaded once)
embedding = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

try:
    loader = TextLoader("../data/acdc.txt")
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = text_splitter.split_documents(docs)
    db = Chroma.from_documents(documents, embedding)
    db._persist_directory
except Exception as e:
    raise RuntimeError(f"Failed to load Chroma DB: {e}")

In [20]:
db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding
)


TypeError: VectorStore.add_documents() missing 1 required positional argument: 'documents'

In [17]:
retriever = db.as_retriever()

In [18]:
retriever.invoke("founders of ac/dc")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Document(id='6aaba6b8-d6a0-4548-996d-e9c0ff5eb8e2', metadata={'source': '../data/acdc.txt'}, page_content="AC/DC were founded by brothers Angus (lead guitar) and Malcolm Young (rhythm guitar), with Colin Burgess (drums), Larry Van Kriedt (bass guitar) and Dave Evans (lead vocals). They underwent several line-up changes before releasing their debut Australasian-only album, High Voltage (1975). Membership stabilised after the release of Let There Be Rock (1977), with the Young brothers, Phil Rudd on drums, Cliff Williams on bass guitar and Bon Scott on lead vocals. Seven months after the release of Highway to Hell (1979), Scott died of alcohol poisoning and English singer Brian Johnson was then recruited as their new frontman. Their first album with Johnson, Back in Black (1980), dedicated to Scott's memory, became the second best-selling album of all time. Their eighth studio album, For Those About to Rock (1981), was their first album to reach number one on the Billboard 200. Rudd was